In [ ]:
!pip install fuzzywuzzy -q

In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd
from tqdm.notebook import tqdm

In [ ]:
# read the CSV
# it's a distinct 'categories' from yelp business dataset
df = pd.read_csv('/content/drive/MyDrive/Personal/Personal Project/ELT YELP Business Category Mapping/business_category_unique_83k.csv')
print(df.isnull().sum())
print(df.info())

categories    0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83160 entries, 0 to 83159
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   categories  83160 non-null  object
dtypes: object(1)
memory usage: 649.8+ KB
None


## **POC Fuzzy Wuzzy**

*Between a list of specific business category, with 11 list of general business category. Which general category is the best match for that list of specific business category?*

In [ ]:
spec_cat_1 = ["Accessories", "Fashion", "Shopping", "Shopping Centers", "Jewelry"] # list of specific business category
query = "Shopping & Retail" # a general business category
similarity_score = fuzz.ratio(spec_cat_1, query) # calculate similarity score

# similarity between a list of specific business categories with a general business category
print(f"Similarity between '{spec_cat_1}' and '{query}': {similarity_score}")

Similarity between '['Accessories', 'Fashion', 'Shopping', 'Shopping Centers', 'Jewelry']' and 'Shopping & Retail': 28


In [ ]:
# list of general business category
general_cat_options = ["Restaurants & Food", "Automotive", "Home Services & Real Estate", "Shopping & Retail",
                 "Beauty & Wellness", "Nightlife & Entertainment",
                 "Hotels & Travel", "Event Planning", "Professional & Local Services",
                 "Active Life & Recreation"]

In [ ]:
# EXAMPLE

similarity_scores = []
for query_option in general_cat_options:
  score = fuzz.ratio(spec_cat_1, query_option)
  similarity_scores.append((spec_cat_1, query_option, score))

# sort the results in descending order of similarity score
similarity_scores.sort(key=lambda x: x[2], reverse=True)

# display the sorted results
for score in similarity_scores:
  print(f"'{score[0]}' vs '{score[1]}': {score[2]}")

# print the general category with the highest score
general_category = max(similarity_scores, key=lambda x: x[2])
print(f"\nThe general category with the highest score is '{general_category[1]}' with a score of {general_category[2]}.")

'['Accessories', 'Fashion', 'Shopping', 'Shopping Centers', 'Jewelry']' vs 'Shopping & Retail': 28
'['Accessories', 'Fashion', 'Shopping', 'Shopping Centers', 'Jewelry']' vs 'Restaurants & Food': 18
'['Accessories', 'Fashion', 'Shopping', 'Shopping Centers', 'Jewelry']' vs 'Nightlife & Entertainment': 17
'['Accessories', 'Fashion', 'Shopping', 'Shopping Centers', 'Jewelry']' vs 'Event Planning': 17
'['Accessories', 'Fashion', 'Shopping', 'Shopping Centers', 'Jewelry']' vs 'Active Life & Recreation': 17
'['Accessories', 'Fashion', 'Shopping', 'Shopping Centers', 'Jewelry']' vs 'Home Services & Real Estate': 15
'['Accessories', 'Fashion', 'Shopping', 'Shopping Centers', 'Jewelry']' vs 'Hotels & Travel': 14
'['Accessories', 'Fashion', 'Shopping', 'Shopping Centers', 'Jewelry']' vs 'Professional & Local Services': 8
'['Accessories', 'Fashion', 'Shopping', 'Shopping Centers', 'Jewelry']' vs 'Beauty & Wellness': 7
'['Accessories', 'Fashion', 'Shopping', 'Shopping Centers', 'Jewelry']' vs 'Au

## **Preprocessing the CSV**

1. Ensure the categories column, free from null
2. Duplicate the column
3. For each list element in all rows, ensure the dtype is string
4. Save it in list format

In [ ]:
print(df['categories'].isnull().sum())
print(df['categories'].duplicated().sum())

0
0


In [ ]:
df['categories_processed'] = df['categories'].copy()
df['categories_processed'] = df['categories_processed'].astype(str)
df['categories_processed'] = df['categories_processed'].str.split(', ')
df['categories_processed'] = df['categories_processed'].apply(lambda x: [str(item) for item in x])
df['categories_processed'][:5]

,categories_processed
0,"[Packing Supplies, Local Services, Home Servic..."
1,"[Masonry/Concrete, Landscaping, Home Services,..."
2,"[Home Services, Movers]"
3,"[Home Services, Home Cleaning]"
4,"[Shopping, Musical Instruments & Teachers]"


In [ ]:
# testing

spec_cat_3 = df['categories_processed'][78888]

similarity_scores = []
for query_option in general_cat_options:
  score = fuzz.ratio(spec_cat_3, query_option)
  similarity_scores.append((spec_cat_3, query_option, score))

# sort the results in descending order of similarity score
similarity_scores.sort(key=lambda x: x[2], reverse=True)

# display the sorted results
for score in similarity_scores:
  print(f"'{score[0]}' vs '{score[1]}': {score[2]}")

# print the general category with the highest score
general_category = max(similarity_scores, key=lambda x: x[2])
print(f"\nThe general category with the highest score is '{general_category[1]}' with a score of {general_category[2]}.")

'['Auto Glass Services', 'Tires', 'Automotive', 'Auto Repair']' vs 'Home Services & Real Estate': 34
'['Auto Glass Services', 'Tires', 'Automotive', 'Auto Repair']' vs 'Automotive': 28
'['Auto Glass Services', 'Tires', 'Automotive', 'Auto Repair']' vs 'Active Life & Recreation': 28
'['Auto Glass Services', 'Tires', 'Automotive', 'Auto Repair']' vs 'Professional & Local Services': 27
'['Auto Glass Services', 'Tires', 'Automotive', 'Auto Repair']' vs 'Nightlife & Entertainment': 19
'['Auto Glass Services', 'Tires', 'Automotive', 'Auto Repair']' vs 'Restaurants & Food': 18
'['Auto Glass Services', 'Tires', 'Automotive', 'Auto Repair']' vs 'Shopping & Retail': 18
'['Auto Glass Services', 'Tires', 'Automotive', 'Auto Repair']' vs 'Hotels & Travel': 18
'['Auto Glass Services', 'Tires', 'Automotive', 'Auto Repair']' vs 'Beauty & Wellness': 15
'['Auto Glass Services', 'Tires', 'Automotive', 'Auto Repair']' vs 'Event Planning': 13

The general category with the highest score is 'Home Services &

## **Find the best match general business category**

1. Create a script to calculate the similarity score for each general business category, with each list. Then store in new columns (only take the business category with the highest similarity score): general_category, similarity score
2. Recheck the distribution & validity before saving the CSV

In [ ]:
def find_best_match(specific_categories, general_categories):
  """
  Finds the best matching general category for a list of specific categories.

  Args:
    specific_categories: A list of specific business categories.
    general_categories: A list of general business categories.

  Returns:
    A tuple containing the best matching general category and its similarity score.
  """
  best_match = None
  highest_score = -1

  for general_category in general_categories:
    score = fuzz.ratio(str(specific_categories), general_category) # convert list to string for fuzz.ratio
    if score > highest_score:
      highest_score = score
      best_match = general_category

  return best_match, highest_score

In [ ]:
tqdm.pandas()

# take 100 sample, to know how is it going
df_sample = df.sample(n=100, random_state=42)

# apply the function to each row of the DataFrame sample
df_sample[['general_category', 'similarity_score']] = df_sample['categories_processed'].progress_apply(
    lambda x: pd.Series(find_best_match(x, general_cat_options))
)

# display the first 10 rows with the new columns
display(df_sample[:10])

  0%|          | 0/100 [00:00<?, ?it/s]

,categories,categories_processed,general_category,similarity_score
47942,"Home & Garden, Shopping, Nurseries & Gardening...","[Home & Garden, Shopping, Nurseries & Gardenin...",Professional & Local Services,27
75294,"Spray Tanning, Hair Removal, Eyelash Service, ...","[Spray Tanning, Hair Removal, Eyelash Service,...",Home Services & Real Estate,23
38145,"Used Car Dealers, Automotive, Car Buyers, Car ...","[Used Car Dealers, Automotive, Car Buyers, Car...",Active Life & Recreation,24
34302,"Nightlife, Bars, Sports Bars, Hotels & Travel,...","[Nightlife, Bars, Sports Bars, Hotels & Travel...",Home Services & Real Estate,27
16314,"Beauty & Spas, Health & Medical, Massage, Mass...","[Beauty & Spas, Health & Medical, Massage, Mas...",Beauty & Wellness,28
8317,"Kitchen & Bath, Shopping, Home & Garden, Home ...","[Kitchen & Bath, Shopping, Home & Garden, Home...",Home Services & Real Estate,31
54906,"Fitness/Exercise Equipment, Sporting Goods, Sh...","[Fitness/Exercise Equipment, Sporting Goods, S...",Restaurants & Food,23
38618,"Food, Breakfast & Brunch, Bakeries, Bagels, Sa...","[Food, Breakfast & Brunch, Bakeries, Bagels, S...",Home Services & Real Estate,26
26018,"Shopping, Knitting Supplies, Arts & Crafts, Ho...","[Shopping, Knitting Supplies, Arts & Crafts, H...",Shopping & Retail,29
82372,"Active Life, Trainers, Fitness & Instruction, ...","[Active Life, Trainers, Fitness & Instruction,...",Active Life & Recreation,31


In [ ]:
tqdm.pandas()

# apply to the whole dataset
df[['general_category', 'similarity_score']] = df['categories_processed'].progress_apply(
    lambda x: pd.Series(find_best_match(x, general_cat_options))
)

# display the first few rows with the new columns
display(df[78500:78520])

  0%|          | 0/83160 [00:00<?, ?it/s]

,categories,categories_processed,general_category,similarity_score
78500,"Skin Care, Waxing, Specialty Schools, Cosmetol...","[Skin Care, Waxing, Specialty Schools, Cosmeto...",Professional & Local Services,26
78501,"Local Flavor, Coffee & Tea, Breakfast & Brunch...","[Local Flavor, Coffee & Tea, Breakfast & Brunc...",Restaurants & Food,16
78502,"Mattresses, Home & Garden, Furniture Stores, S...","[Mattresses, Home & Garden, Furniture Stores, ...",Professional & Local Services,25
78503,"Beauty & Spas, Medical Spas, Laser Hair Remova...","[Beauty & Spas, Medical Spas, Laser Hair Remov...",Professional & Local Services,23
78504,"Personal Chefs, Event Planning & Services, Spe...","[Personal Chefs, Event Planning & Services, Sp...",Home Services & Real Estate,23
78505,"Restaurants, Malaysian, Food, Vegetarian, Brea...","[Restaurants, Malaysian, Food, Vegetarian, Bre...",Restaurants & Food,28
78506,"Pet Services, Pet Cremation Services, Pets","[Pet Services, Pet Cremation Services, Pets]",Home Services & Real Estate,42
78507,"Waxing, Hair Removal, Hair Stylists, Hair Exte...","[Waxing, Hair Removal, Hair Stylists, Hair Ext...",Professional & Local Services,27
78508,"Fitness & Instruction, Gyms, Active Life, Cycl...","[Fitness & Instruction, Gyms, Active Life, Cyc...",Active Life & Recreation,33
78509,"Auto Parts & Supplies, Health & Medical, Optom...","[Auto Parts & Supplies, Health & Medical, Opto...",Home Services & Real Estate,22


if you look at the displayed result, sometimes the result is less accurate. For example:

[Waxing, Hair Removal, Hair Salons, Hair Stylists, Beauty & Spas] >> Categorized as "Restaurants & Food". While it should be "Beauty & Wellness"

In [ ]:
df['general_category'].value_counts()

,count
general_category,
Restaurants & Food,22221
Home Services & Real Estate,20400
Professional & Local Services,17432
Nightlife & Entertainment,8305
Active Life & Recreation,5352
Shopping & Retail,4691
Beauty & Wellness,2205
Hotels & Travel,1004
Automotive,844


In [ ]:
df = df.drop(columns=['categories_processed'], axis=1)
df.columns

Index(['categories', 'general_category', 'similarity_score'], dtype='object')

In [ ]:
df.to_csv('business category mapping fuzzywuzzy NEW.csv', index=False)